# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e compramos e vendemos commodities:
- Soja, Milho, Trigo, Petróleo, etc.

Precisamos pegar na internet, de forma automática, a cotação de todas as commodites e ver se ela está abaixo do nosso preço ideal de compra. Se tiver, precisamos marcar como uma ação de compra para a equipe de operações.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=share_link

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [1]:
#!pip install selenium


In [2]:
# passo 1  : Abrir o navegador

from selenium import webdriver

navegador = webdriver.Chrome()
navegador.get("https://google.com") # .get significa entrar em um site
# Passo a passo




In [3]:
# passo 2 : importar a base de dados
import pandas as pd
tabela = pd.read_excel("commodities.xlsx")
display(tabela)


,Produto,Preço Ideal,Preço Atual,Comprar
0,Milho,85.32,NaN,NaN
1,Soja,163.59,NaN,NaN
2,Boi,282.20,NaN,NaN
3,Petróleo,424.37,NaN,NaN
4,Algodão,497.76,NaN,NaN
5,Açúcar,136.23,NaN,NaN
6,Café,1092.87,NaN,NaN
7,Ouro,321.77,NaN,NaN
8,Trigo,1549.11,NaN,NaN
9,Tilápia,9.05,NaN,NaN


In [4]:
# passo 3 : para cada produto da base de dados

# passo 4 : pesquisar o preço do produto

# import unicodedata
# produto = unicodedata.normalize("NFKD", produto).encode("ascii","ignore") -> outra forma de trocar as letras com acento.

for linha in tabela.index:
    produto = tabela.loc[linha, "Produto"] # .loc localiza a "linha" e a "coluna" da tabela de itens
    print(produto)
    
    produto = produto.replace("ó","o").replace("ã","a").replace("é","e").replace("á","a").replace("ç","c").replace("ú","u") # o replace() troca as letras com acento para não dar erro na hora de entrar nos endereços do site.

    link = f"https://www.melhorcambio.com/{produto}-hoje"   # criou uma varável link que recebe o endereço: site melhor cambio com a cotação do milho de hoje.
    print(link)
    navegador.get(link)       # usa o comando .get() junto com a variável navegador para entrar no site melhor cambio recebido pela variável link.

# Passo 5 : atualizar o preço na base de dados.

    preco = navegador.find_element('xpath', '//*[@id="comercial"]').get_attribute('value')
    preco = preco.replace(".","").replace(",",".") # aqui estou mudando o formato brasileiro para americano da coluna "Preço atual" para colocar só . nas casas decimais e retirar o . das de milhar.
    print(preco)
    tabela.loc[linha,"Preço Atual"] = float(preco)  # tabela.loc[liha,"Preço Atual"] recebe as informações da variável "preço" para preencher a coluna "Preço Atual". obs: coloquei o "float" para garantir que ela transforme os valores em números realmente.
    
print("Acabou")
display(tabela)

# O que você pode fazer quando encontrar os valores
#----------------------------------------------------

# 1  .click() é igual a "clicar"

# 2  .send_keys("texto") -> escrever algo

# 3  .get_attribute()  -> pegar um valor



Milho
https://www.melhorcambio.com/Milho-hoje
54.27
Soja
https://www.melhorcambio.com/Soja-hoje
132.34
Boi
https://www.melhorcambio.com/Boi-hoje
241.75
Petróleo
https://www.melhorcambio.com/Petroleo-hoje
371.90
Algodão
https://www.melhorcambio.com/Algodao-hoje
0.00
Açúcar
https://www.melhorcambio.com/Acucar-hoje
148.23
Café
https://www.melhorcambio.com/Cafe-hoje
992.80
Ouro
https://www.melhorcambio.com/Ouro-hoje
320.24
Trigo
https://www.melhorcambio.com/Trigo-hoje
0.00
Tilápia
https://www.melhorcambio.com/Tilapia-hoje
9.84
Acabou


,Produto,Preço Ideal,Preço Atual,Comprar
0,Milho,85.32,54.27,NaN
1,Soja,163.59,132.34,NaN
2,Boi,282.20,241.75,NaN
3,Petróleo,424.37,371.90,NaN
4,Algodão,497.76,0.00,NaN
5,Açúcar,136.23,148.23,NaN
6,Café,1092.87,992.80,NaN
7,Ouro,321.77,320.24,NaN
8,Trigo,1549.11,0.00,NaN
9,Tilápia,9.05,9.84,NaN


In [5]:
# Passo 6 : decidir quais preços comprar preenchendo a coluna comprar.

tabela["Comprar"] = tabela["Preço Atual"] < tabela["Preço Ideal"] # aqui é comparado se o "preço atual" é menor que o "preço ideal" para que a compra aconteça.
#tabela["Não comprar"] = tabela["Preço Atual"] > tabela["Preço Ideal"]
display(tabela)

# Passo 7 : exportar a tabela para uma planilha.

tabela.to_excel("commodities_atualizado.xlsx", index=False) # .to exporta a tabela para uma planilha excel. obs: "index=False" retira a coluna de indices da tabela.


,Produto,Preço Ideal,Preço Atual,Comprar
0,Milho,85.32,54.27,True
1,Soja,163.59,132.34,True
2,Boi,282.20,241.75,True
3,Petróleo,424.37,371.90,True
4,Algodão,497.76,0.00,True
5,Açúcar,136.23,148.23,False
6,Café,1092.87,992.80,True
7,Ouro,321.77,320.24,True
8,Trigo,1549.11,0.00,True
9,Tilápia,9.05,9.84,False


In [6]:
navegador.quit() # o .quit() fecha automaticamento o navegador.